In [14]:
!pip install groq python-dotenv

In [15]:
from getpass import getpass
import os
from groq import Groq

# Secure input (hidden while typing)
api_key = getpass("Enter your Groq API Key: ")

# Initialize Groq client
client = Groq(api_key=api_key)

print("✅ Groq client initialized successfully!")

Enter your Groq API Key: ··········
✅ Groq client initialized successfully!


In [16]:
MODEL_NAME = "llama-3.3-70b-versatile"
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a highly rigorous technical support AI.
Focus on debugging, stack traces, and programming issues.
Be structured, precise, and provide code examples when helpful."""
    },

    "billing": {
        "system_prompt": """You are a polite and empathetic billing specialist.
Handle subscription issues, refunds, and payment concerns.
Be reassuring and clearly explain policies and next steps."""
    },

    "general": {
        "system_prompt": """You are a friendly general assistant.
Handle casual conversation and general knowledge queries."""
    },

    "tool": {
        "system_prompt": """You are a financial data assistant."""
    }
}

In [17]:
def route_prompt(user_input):

    router_prompt = f"""
Classify the following user input into one of these categories:
[technical, billing, general, tool]

Use "tool" if the user is asking for real-time data like cryptocurrency prices.

Return ONLY the category name.

User Input:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a strict classification engine."},
            {"role": "user", "content": router_prompt}
        ],
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()

    # Safety fallback
    if category not in MODEL_CONFIG:
        return "general"

    return category

In [18]:
import random

def get_bitcoin_price():
    # Mock real-time price
    price = random.randint(40000, 70000)
    return f"The current Bitcoin price is approximately ${price}."

In [19]:
def process_request(user_input):

    # Step 1: Route the query
    category = route_prompt(user_input)
    print(f"[Router Decision]: {category}")

    # Step 2: If tool category → use function
    if category == "tool":
        return get_bitcoin_price()

    # Step 3: Load expert system prompt
    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    # Step 4: Call expert LLM
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content

In [20]:
print(process_request("My Python script throws an IndexError on line 5."))

[Router Decision]: technical
**Debugging IndexError in Python**

To effectively debug the IndexError in your Python script, I'll need more information about the code. However, I can guide you through a general approach to identify and fix the issue.

### Steps to Debug IndexError:

1. **Review the code**: Check the line of code where the error occurs (line 5 in your case). Look for any indexing operations, such as accessing elements in a list or tuple.
2. **Check the data**: Verify that the data you're trying to access is valid and within the expected bounds. Make sure the list or tuple has enough elements to access the index specified.
3. **Print or log variables**: Temporarily add print statements or use a debugger to inspect the values of variables involved in the indexing operation. This can help you understand what's causing the error.

### Example Code:
```python
# Example list
my_list = [1, 2, 3]

# Error-prone code (IndexError will occur)
print(my_list[3])  # IndexError: list i

In [21]:
print(process_request("I was charged twice for my subscription this month."))

[Router Decision]: billing
I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you. Please know that I'm here to help you resolve this issue as quickly and efficiently as possible.

To get started, could you please provide me with a little more information about the duplicate charge? For example, what is the date of the duplicate charge, and what is the amount that was charged? Additionally, have you checked your email to see if you received two separate confirmation emails for the charges?

Once I have this information, I'll be happy to look into this further and work with you to correct the issue. We have a process in place to handle duplicate charges, and I'll do my best to ensure that you're refunded the incorrect amount as soon as possible.

Please rest assured that we take situations like this very seriously, and I'm committed to making things right. Your satisfaction is our top priority, and I apprecia

In [22]:
print(process_request("Tell me something interesting about space."))

[Router Decision]: general
Did you know that there's a giant storm on Jupiter that's been raging for at least 150 years? It's called the Great Red Spot, and it's a massive anticyclonic storm (meaning it's a high-pressure region with clockwise rotation) that's larger than Earth in diameter.

The storm is so big that three Earths could fit inside it, and it's been continuously raging since at least 1831, when it was first observed. The storm is thought to be a persistent anticyclonic storm, similar to a hurricane on Earth, but it's much larger and more stable.

The Great Red Spot is also incredibly tall, with cloud tops reaching heights of up to 12 km (7.5 miles) above the surrounding clouds. The storm's distinctive red color is thought to be due to the presence of sulfur and phosphorus compounds in the clouds.

Despite being a massive storm, the Great Red Spot is actually shrinking, and astronomers are still trying to understand the underlying mechanisms that drive its behavior. Isn't i

In [23]:
print(process_request("What is the current price of Bitcoin?"))

[Router Decision]: tool
The current Bitcoin price is approximately $40565.
